## Miscellaneous notes/thoughts

### 10/6/21 Thoughts

**Reason for prioritizing the draft**
To win, you need to have the best team. Working the waivers helps a lot, but most of the players who end up moving the needle for your team will be drafted. Therefore, it make sense to focus on a draft-only model for now.

**Thesis**: To have the best draft, you need to get the right player at the right time. The #1 QB will probably score the most points in any given season, there are many more viable QB options than there are viable RB/WR options.

As a result, this will need to be a combination of WAR vs. ADP, with some balance between overall WAR and positional scarcity.

We can add in sentiment analysis for estimating changes to ADP (and can also include an Eagles bias). We'll also want to update this to remove people with unexpected late injuries.

## Next Steps

Finish processing positional data

## The Code

In [1]:
import numpy as np
import pandas as pd

## Initial Player Pool

### Quarterbacks

In [27]:
qbs = pd.read_csv("C:\\Users\\pauls\\Desktop\\data_science\\fantasy_football\\QB_Projections_FantasyPros_2021.csv")

In [28]:
qbs = qbs.rename(columns={
    'ATT': 'Pass_Att',
    'CMP': 'Cmp',
    'YDS': 'Pass_Yds',
    'TDS': 'Pass_TDs',
    'ATT.1': 'Rush_Att',
    'YDS.1': 'Rush_Yds',
    'TDS.1': 'Rush_TDs',
    'FL': 'Fumbles_Lost'
})

In [29]:
qbs = qbs.set_index('Player')
qbs.head()

,Team,Pass_Att,Cmp,Pass_Yds,Pass_TDs,INTS,Rush_Att,Rush_Yds,Rush_TDs,Fumbles_Lost,FPTS
Player,,,,,,,,,,,
Josh Allen,BUF,595.4,394.6,"4,574.90",33.3,12.9,109.3,517.3,7.2,4.3,389.3
Patrick Mahomes II,KC,620.8,413.1,"5,027.60",38.0,9.5,55.5,289.8,2.8,1.8,386.0
Kyler Murray,ARI,585.6,389.0,"4,261.80",27.6,12.7,128.1,726.5,7.3,2.8,379.4
Lamar Jackson,BAL,462.7,298.5,"3,406.20",28.1,11.5,154.5,952.7,7.3,3.4,369.2
Dak Prescott,DAL,623.9,418.7,"4,886.10",31.5,11.3,64.1,298.1,4.3,3.0,359.4


We likely only care about the first 30 QBs

In [30]:
qbs = qbs[0:31]

We need to convert numeric quantities to floats

In [31]:
pass_yds = []

for player, row in qbs.iterrows():
    pass_yds.append(float(str(row['Pass_Yds']).replace(',', '')))    

In [32]:
qbs['Pass_Yds'] = pass_yds
qbs.head()

,Team,Pass_Att,Cmp,Pass_Yds,Pass_TDs,INTS,Rush_Att,Rush_Yds,Rush_TDs,Fumbles_Lost,FPTS
Player,,,,,,,,,,,
Josh Allen,BUF,595.4,394.6,4574.9,33.3,12.9,109.3,517.3,7.2,4.3,389.3
Patrick Mahomes II,KC,620.8,413.1,5027.6,38.0,9.5,55.5,289.8,2.8,1.8,386.0
Kyler Murray,ARI,585.6,389.0,4261.8,27.6,12.7,128.1,726.5,7.3,2.8,379.4
Lamar Jackson,BAL,462.7,298.5,3406.2,28.1,11.5,154.5,952.7,7.3,3.4,369.2
Dak Prescott,DAL,623.9,418.7,4886.1,31.5,11.3,64.1,298.1,4.3,3.0,359.4


Creating a list to store points for our league's scoring system

In [33]:
player_points = []
for player in qbs.index:
    points = ((.04 * qbs['Pass_Yds'][player])
            + (4 * qbs['Pass_TDs'][player])
            - (2 * qbs['INTS'][player])
            + (.1 * qbs['Rush_Yds'][player])
            + (6 * qbs['Rush_TDs'][player])
            - (2 * qbs['Fumbles_Lost'][player]))
    
    points = round(points, 2)

    player_points.append(points)
    
player_points[0:5]

[376.73, 376.28, 366.32, 357.92, 348.45]

In [34]:
qbs['FPTS'] = player_points

qbs.head()

,Team,Pass_Att,Cmp,Pass_Yds,Pass_TDs,INTS,Rush_Att,Rush_Yds,Rush_TDs,Fumbles_Lost,FPTS
Player,,,,,,,,,,,
Josh Allen,BUF,595.4,394.6,4574.9,33.3,12.9,109.3,517.3,7.2,4.3,376.73
Patrick Mahomes II,KC,620.8,413.1,5027.6,38.0,9.5,55.5,289.8,2.8,1.8,376.28
Kyler Murray,ARI,585.6,389.0,4261.8,27.6,12.7,128.1,726.5,7.3,2.8,366.32
Lamar Jackson,BAL,462.7,298.5,3406.2,28.1,11.5,154.5,952.7,7.3,3.4,357.92
Dak Prescott,DAL,623.9,418.7,4886.1,31.5,11.3,64.1,298.1,4.3,3.0,348.45


### Running Backs

In [35]:
rbs = pd.read_csv("C:\\Users\\pauls\\Desktop\\data_science\\fantasy_football\\RB_Projections_FantasyPros_2021.csv")

In [36]:
rbs.head()

,Player,Team,ATT,YDS,TDS,REC,YDS.1,TDS.1,FL,FPTS
0,Christian McCaffrey,CAR,293.9,"1,267.90",11.6,92.7,755.8,4.4,1.7,295.3
1,Dalvin Cook,MIN,312.7,"1,456.30",12.9,55.6,476.5,2.1,2.6,278.6
2,Derrick Henry,TEN,350.1,"1,724.30",14.0,24.8,188.1,1.0,2.6,276.2
3,Alvin Kamara,NO,208.5,"1,008.50",10.4,82.0,703.9,3.4,1.5,251.2
4,Aaron Jones,GB,231.1,"1,145.90",10.0,54.4,413.5,2.8,1.6,229.5


In [37]:
rbs = rbs.rename(columns={
    'YDS': 'Rush_Yds',
    'TDS': 'Rush_TDs',
    'YDS.1': 'Rec_Yds',
    'TDS.1': 'Rec_TDs',
    'FL': 'Fumbles_Lost'
})

We'll grab the first 75 for now

In [38]:
rbs = rbs[:75]

In [39]:
rush_yds = []
rec_yds = []

for player, row in rbs.iterrows():
    rush_yds.append(float(str(row['Rush_Yds']).replace(',', '')))  
    rec_yds.append(float(str(row['Rec_Yds']).replace(',', '')))

In [41]:
rbs.head()

,Player,Team,ATT,Rush_Yds,Rush_TDs,REC,Rec_Yds,Rec_TDs,Fumbles_Lost,FPTS
0,Christian McCaffrey,CAR,293.9,"1,267.90",11.6,92.7,755.8,4.4,1.7,295.3
1,Dalvin Cook,MIN,312.7,"1,456.30",12.9,55.6,476.5,2.1,2.6,278.6
2,Derrick Henry,TEN,350.1,"1,724.30",14.0,24.8,188.1,1.0,2.6,276.2
3,Alvin Kamara,NO,208.5,"1,008.50",10.4,82.0,703.9,3.4,1.5,251.2
4,Aaron Jones,GB,231.1,"1,145.90",10.0,54.4,413.5,2.8,1.6,229.5


In [ ]:
rb_points =[]

### Wide Receivers

In [8]:
wrs = pd.read_csv("C:\\Users\\pauls\\Desktop\\data_science\\fantasy_football\\WR_Projections_FantasyPros_2021.csv")

In [15]:
wrs.head()

,Player,Team,REC,YDS,TDS,ATT,YDS.1,TDS.1,FL,FPTS
0,Tyreek Hill,KC,102.0,"1,457.50",12.0,13.1,83.7,0.9,0.5,230.6
1,Davante Adams,GB,124.2,"1,485.30",12.8,0.0,0.0,0.0,0.5,224.4
2,Stefon Diggs,BUF,115.1,"1,490.90",9.5,2.1,13.7,0.0,0.8,206.3
3,Calvin Ridley,ATL,101.0,"1,441.40",9.0,3.9,22.6,0.1,0.5,199.7
4,Justin Jefferson,MIN,96.2,"1,434.30",8.1,0.7,3.8,0.0,0.5,191.3


### Tight Ends

In [9]:
tes = pd.read_csv("C:\\Users\\pauls\\Desktop\\data_science\\fantasy_football\\TE_Projections_FantasyPros_2021.csv")

In [16]:
tes.head()

,Player,Team,REC,YDS,TDS,FL,FPTS
0,Travis Kelce,KC,112.3,"1,405.20",10.4,0.5,201.7
1,Darren Waller,LV,104.0,"1,209.10",7.9,0.6,167.1
2,George Kittle,SF,92.0,"1,138.70",6.5,0.4,152.0
3,Mark Andrews,BAL,72.9,878.4,8.0,0.6,134.8
4,T.J. Hockenson,DET,75.5,838.3,5.3,0.5,114.5


### Defense/Special Teams

In [11]:
dsts = pd.read_csv("C:\\Users\\pauls\\Desktop\\data_science\\fantasy_football\\DST_Projections_FantasyPros_2021.csv")

In [17]:
dsts.head()

,Player,Team,SACK,INT,FR,FF,TD,SAFETY,PA,YDS_AGN,FPTS
0,Tampa Bay Buccaneers,NaN,46.5,15.8,11.1,17.4,2.8,0.9,366.0,"5,515.50",119.2
1,Pittsburgh Steelers,NaN,47.1,14.9,10.4,15.7,2.7,0.9,358.5,"5,547.40",115.4
2,Los Angeles Rams,NaN,45.9,14.4,10.7,15.8,2.7,0.9,366.0,"5,581.50",113.9
3,Washington Football Team,NaN,46.3,14.3,10.5,15.6,2.4,0.5,373.0,"5,604.80",111.3
4,Indianapolis Colts,NaN,42.2,14.1,10.6,17.1,2.9,0.9,382.6,"5,784.10",110.6


### Kickers

In [2]:
kickers = pd.read_csv("C:\\Users\\pauls\\Desktop\\data_science\\fantasy_football\\K_Projections_FantasyPros_2021.csv")

In [4]:
kickers.head()

,Player,Team,FG,FGA,XPT,FPTS
0,Justin Tucker,BAL,28.3,32.6,49.5,134.4
1,Harrison Butker,KC,28.8,33.3,47.1,133.4
2,Greg Zuerlein,DAL,28.0,35.9,44.5,128.6
3,Tyler Bass,BUF,26.9,33.0,46.9,127.6
4,Ryan Succop,TB,26.3,31.6,48.3,127.1


Overall

## Final function for who to draft

In [2]:
def who_should_i_draft(last_selected_player, position_of_need=''):
    """Return the player to draft next based on the last selected player."""
    return player_to_draft

## Function for my current team

In [ ]:
def update_team(drafted_player):
    """Updates team info based on the most recently drafted player."""
    # Get the player's position
    
    # Update team position construction
        # If quarterback, update the QB count
        # Else if RB, update the RB count
        # Else if WR, update the WR count
        # Else if TE, update the TE count
        # Else if D/ST, update the D/ST count
        # Else update the K count
        
    # Update team's overall WAR
    
    # Update team's positional WAR

## Sentiment analysis for ADP adjustment

Here, we will extract recent articles about players and return scaled values to adjust their expected ADP accordingly.